## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-12-15-08-49 +0000,bbc,Bill to lower voting age to 16 introduced in P...,https://www.bbc.com/news/articles/cly2jjvx466o...
1,2026-02-12-15-07-14 +0000,bbc,Starmer agrees with Nandy over government's ch...,https://www.bbc.com/news/articles/czj17w47y4jo...
2,2026-02-12-15-04-47 +0000,nypost,Magnitude 2.5 earthquake rattles South Carolin...,https://nypost.com/2026/02/12/us-news/magnitud...
3,2026-02-12-15-00-00 +0000,wsj,"In Miami Beach, Fla., a flood-resistant home u...",https://www.wsj.com/real-estate/luxury-homes/f...
4,2026-02-12-15-00-00 +0000,wsj,Josh Gruenbaum’s brash ways as a federal agenc...,https://www.wsj.com/politics/national-security...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2518/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
43,trump,38
107,epstein,18
50,he,16
581,new,14
70,will,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
64,2026-02-12-12-06-00 +0000,wsj,California Gov. Gavin Newsom is on the road ag...,https://www.wsj.com/politics/gavin-newsom-the-...,136
371,2026-02-11-15-19-00 +0000,wsj,House lawmakers rejected an attempt by Speaker...,https://www.wsj.com/politics/policy/house-reje...,135
187,2026-02-12-00-44-00 +0000,wsj,The GOP-led House passed a resolution designed...,https://www.wsj.com/politics/policy/gop-led-ho...,116
6,2026-02-12-14-52-00 +0000,wsj,Border czar Tom Homan said the Trump administr...,https://www.wsj.com/politics/policy/tom-homan-...,101
203,2026-02-11-23-37-50 +0000,nypost,Six House Republicans break with White House t...,https://nypost.com/2026/02/11/us-news/six-hous...,91


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
64,136,2026-02-12-12-06-00 +0000,wsj,California Gov. Gavin Newsom is on the road ag...,https://www.wsj.com/politics/gavin-newsom-the-...
11,78,2026-02-12-14-38-32 +0000,nypost,Woman seen in photo under disgraced ex-Prince ...,https://nypost.com/2026/02/12/us-news/woman-in...
187,73,2026-02-12-00-44-00 +0000,wsj,The GOP-led House passed a resolution designed...,https://www.wsj.com/politics/policy/gop-led-ho...
334,65,2026-02-11-17-27-07 +0000,nypost,Ex-Google CEO Eric Schmidt spent New Year’s wi...,https://nypost.com/2026/02/11/business/ex-goog...
311,52,2026-02-11-18-53-53 +0000,nypost,"Jesse Van Rootselaar, 18, ID’d as transgender ...",https://nypost.com/2026/02/11/world-news/tumbl...
164,51,2026-02-12-02-48-00 +0000,wsj,The Pentagon’s use of a high-energy laser to c...,https://www.wsj.com/us-news/faa-halts-el-paso-...
175,48,2026-02-12-02-02-00 +0000,wsj,Demand for healthcare workers is outstripping ...,https://www.wsj.com/economy/jobs/healthcare-jo...
16,46,2026-02-12-14-32-16 +0000,nypost,Border czar Tom Homan announces end of ICE sur...,https://nypost.com/2026/02/12/us-news/tom-homa...
56,39,2026-02-12-12-26-06 +0000,nypost,Key giveaways in Nancy Guthrie case that show ...,https://nypost.com/2026/02/12/us-news/key-give...
194,37,2026-02-12-00-01-41 +0000,nypost,Mother killed by transgender Canadian school s...,https://nypost.com/2026/02/11/world-news/mothe...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
